## Evaluation

Batch prediction and evaluation are very similar. They are based on DataFlow pipeline and CloudML provides Evaluate and Prediction DataFlow transform. Datalab can generate DataFlow pipeline code template for you, just like Preprocessing.

Run "%mlalpha evaluate" to generate input cell. After fill in the required fields, it becomes:
```
%%mlalpha evaluate
preprocessed_eval_data_path: /content/datalab/tmp/ml/census/preprocessed/features_eval*
metadata_path: /content/datalab/tmp/ml/census/preprocessed/metadata.yaml
model_dir: /content/datalab/tmp/ml/census/model/model
output_dir: /content/datalab/tmp/ml/census/evaluate
```

In [1]:

# header
"""
Following code is generated from command line:
%%mlalpha evaluate
preprocessed_eval_data_path: /content/datalab/tmp/ml/census/preprocessed/features_eval*
metadata_path: /content/datalab/tmp/ml/census/preprocessed/metadata.yaml
model_dir: /content/datalab/tmp/ml/census/model/model
output_dir: /content/datalab/tmp/ml/census/evaluate

Please modify as appropriate!!!
"""

# imports
import apache_beam as beam
import google.cloud.ml as ml
import google.cloud.ml.analysis as analysis
import google.cloud.ml.io as io
import json
import os

# defines
def extract_values((example, prediction)):
  import tensorflow as tf
  tf_example = tf.train.Example()
  tf_example.ParseFromString(example.values()[0])
  feature_map = tf_example.features.feature
  values = {'target': feature_map['target'].float_list.value[0]}
  values.update(prediction)
  return values

OUTPUT_DIR = '/content/datalab/tmp/ml/census/evaluate'
pipeline = beam.Pipeline('DirectPipelineRunner')


# evaluation

eval_features = (pipeline | 'ReadEval' >> io.LoadFeatures('/content/datalab/tmp/ml/census/preprocessed/features_eval*'))
trained_model = pipeline | 'LoadModel' >> io.LoadModel('/content/datalab/tmp/ml/census/model/model')
evaluations = (eval_features | 'Evaluate' >> ml.Evaluate(trained_model) |
    beam.Map('ExtractEvaluationResults', extract_values))
eval_data_sink = beam.io.TextFileSink(os.path.join(OUTPUT_DIR, 'eval'), shard_name_template='')
evaluations | beam.io.textio.WriteToText(os.path.join(OUTPUT_DIR, 'eval'), shard_name_template='')

# analysis

# run pipeline
pipeline.run()


View the output eval file:

In [2]:
!head /content/datalab/tmp/ml/census/evaluate/eval

{'target': 172300.0, u'key': '705', u'predicted': 62663.512}
{'target': 28000.0, u'key': '1204', u'predicted': 63287.945}
{'target': 37300.0, u'key': '1758', u'predicted': 54482.168}
{'target': 42400.0, u'key': '2534', u'predicted': 44536.602}
{'target': 100000.0, u'key': '3172', u'predicted': 61717.07}
{'target': 53000.0, u'key': '3823', u'predicted': 58346.137}
{'target': 63300.0, u'key': '4245', u'predicted': 59792.566}
{'target': 56200.0, u'key': '4918', u'predicted': 63986.496}
{'target': 132000.0, u'key': '5352', u'predicted': 62873.809}
{'target': 55000.0, u'key': '6111', u'predicted': 53053.559}


To generate a pipeline that runs in cloud, simply add --cloud to "%mlalpha evaluate". Also all paths need to be GCS paths.

In [4]:
import os

bucket = 'gs://' + datalab_project_id() + '-sampledata'
eval_data_path = os.path.join(bucket, 'census', 'preprocessed', 'features_eval')
metadata_path = os.path.join(bucket, 'census', 'preprocessed', 'metadata.yaml')
model_path = os.path.join(bucket, 'census', 'trained', 'model')
output_dir = os.path.join(bucket, 'census', 'evaluate')
eval_file = os.path.join(output_dir, 'eval*')

In [ ]:
%%mlalpha evaluate --cloud
preprocessed_eval_data_path: $eval_data_path
metadata_path: $metadata_path
model_dir: $model_path
output_dir: $output_dir

After you run the generated code, you can go to Developer Console to see the DataFlow job: https://pantheon.corp.google.com/dataflow (and select the right project). Also you can check the results as below:

In [ ]:
!gsutil cat $eval_file | head -5